In [19]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_product_listing(url, num_pages):
    data = []

    for page in range(1, num_pages + 1):
        page_url = url + f'&page={page}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        products = soup.find_all('div', {'data-component-type': 's-search-result'})

        for product in products:
            product_url = product.find('a', {'class': 'a-link-normal'})['href']
            product_name = product.find('span', {'class': 'a-text-normal'}).text.strip()

            product_price = product.find('span', {'class': 'a-offscreen'})
            if product_price:
                product_price = product_price.text.strip()
            else:
                product_price = "N/A"

            rating = product.find('span', {'class': 'a-icon-alt'})
            if rating:
                rating = rating.text.split()[0]
            else:
                rating = "N/A"

            num_reviews = product.find('span', {'class': 'a-size-base'}).text.split()[0]

            data.append({
                'Product URL': product_url,
                'Product Name': product_name,
                'Product Price': product_price,
                'Rating': rating,
                'Number of Reviews': num_reviews
            })

        time.sleep(2)

    return data


base_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
num_pages_to_scrape = 20


scraped_data = scrape_product_listing(base_url, num_pages_to_scrape)


csv_filename = 'scraped_products.csv'
csv_fields = ['Product URL', 'Product Name', 'Product Price', 'Rating', 'Number of Reviews']

with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_fields)
    writer.writeheader()
    for item in scraped_data:
        writer.writerow(item)

print(f"Scraped data saved to {csv_filename}")


Scraped data saved to scraped_products.csv
